In [1]:
from transformers import TransfoXLTokenizer, TFTransfoXLForSequenceClassification
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import model_selection
from datasets import Dataset
from transformers import DataCollatorWithPadding
from transformers import create_optimizer
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from tensorflow.nn import softmax
#from imblearn.over_sampling import RandomOverSampler

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [ ]:
data = pd.read_csv("../../data/sensitivity_data/sensitivity_dataset.csv")
data = data[["Document","Sensitivity"]]
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['Document'],data['Sensitivity'],test_size=0.2,random_state=5)
train_x = np.array(train_x)
train_y = np.array(train_y)

In [ ]:
################################################
## Try to train again with a balanced dataset ##
################################################

data = pd.read_csv("../../data/sensitivity_data/sensitivity_dataset.csv")
data = data[["Document","Sensitivity"]]

train_x, test_x, train_y, test_y = model_selection.train_test_split(data['Document'],data['Sensitivity'],test_size=0.2,random_state=5)
train_x = pd.DataFrame([train_x,train_y]).T

classes_zero = train_x[train_x['Sensitivity'] == 0]
classes_one = train_x[train_x['Sensitivity'] == 1]

classes_zero = classes_zero.sample(len(classes_one))

undersampled_dataset = pd.concat([classes_zero,classes_one])
undersampled_dataset = undersampled_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

train_x = np.array(undersampled_dataset["Document"])
train_y = np.array(undersampled_dataset["Sensitivity"])

In [3]:
############################################################
## Try to train again with a oversampled balanced dataset ##
############################################################

data = pd.read_csv("../../data/sensitivity_data/sensitivity_dataset.csv")
data = data[["Document","Sensitivity"]]

train_x, test_x, train_y, test_y = model_selection.train_test_split(data['Document'],data['Sensitivity'],test_size=0.2,random_state=5)
train_x = pd.DataFrame([train_x,train_y]).T

classes_zero = train_x[train_x['Sensitivity'] == 0]
classes_one = train_x[train_x['Sensitivity'] == 1]
print(len(classes_zero),len(classes_one))

classes_one = classes_one.sample(len(classes_zero), replace=True)

oversampled_dataset = pd.concat([classes_zero,classes_one])
oversampled_dataset = oversampled_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

train_x = np.array(oversampled_dataset["Document"])
train_y = np.array(oversampled_dataset["Sensitivity"])

print(len(classes_zero),len(classes_one))

2632 408
2632 2632


In [4]:
dataset = []
for i in range(len(train_x)):
    dataset.append({"label" : train_y[i], "text" : train_x[i]})

dataset = pd.DataFrame(dataset)
dataset = Dataset.from_pandas(dataset)

def preprocess_function(examples):
    return tokenizer(examples["text"],padding=False,truncation=True)

tokenised_dataset = dataset.map(preprocess_function, batched=False)

100%|██████████| 5264/5264 [00:18<00:00, 285.10ex/s]


In [5]:
tokenised_dataset.features

{'label': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [6]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, return_tensors="tf")

In [7]:
tf_train_dataset = tokenised_dataset.to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'label'],
    label_cols=['label'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

print(len(tf_train_dataset))

329


In [8]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tf_train_dataset) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(
    init_lr=2e-5, 
    num_warmup_steps=0, 
    num_train_steps=total_train_steps
)

In [9]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [10]:
import tensorflow as tf
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [11]:
model.fit(tf_train_dataset,epochs=15)

Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
329/329 [==============================] - 4955s 15s/step - loss: 1.2327 - logits_loss: 0.8636 - loss_loss: 0.3691
Epoch 2/15
329/329 [==============================] - 4904s 15s/step - loss: 1.1784 - logits_loss: 0.8241 - loss_loss: 0.3543
Epoch 

In [12]:
# Make predictions

data = pd.read_csv("../../data/sensitivity_data/sensitivity_dataset.csv")
data = data[["Document","Sensitivity"]]
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['Document'],data['Sensitivity'],test_size=0.2,random_state=5)
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x = np.array(test_x)
test_y = np.array(test_y)

dataset = []
for i in range(len(test_x)):
    dataset.append({"label" : test_y[i], "text" : test_x[i]})

dataset = pd.DataFrame(dataset)
dataset = Dataset.from_pandas(dataset)

def preprocess_function(examples):
    return tokenizer(examples["text"],padding=False,truncation=True)

tokenised_dataset = dataset.map(preprocess_function, batched=True)

tf_test_dataset = tokenised_dataset.to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'label'],
    label_cols=['label'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

100%|██████████| 1/1 [00:04<00:00,  4.34s/ba]


In [13]:
output = model.predict(tf_test_dataset)

In [14]:
prediction_probs = tf.nn.softmax(output.logits)
predictions = []
for prob in prediction_probs:
    if prob[0] > prob[1]:
        predictions.append(0)
    else:
        predictions.append(1)

In [15]:
# None of the predictions are 1
print(sum(predictions))

0
